In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
from catboost import CatBoost as cb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec 
import nltk
import re
from nltk.stem import WordNetLemmatizer

In [2]:
train = pd.read_csv("train_F3WbcTw.csv")
test = pd.read_csv("test_tOlRoBf.csv")
sample = pd.read_csv("sample_submission_i5xnIZD.csv")

In [3]:
train_df = train.copy()
test_df = test.copy()

In [4]:
wordnet_lemmatizer = WordNetLemmatizer()

In [5]:
train_df['text'] = train_df['text'].apply(wordnet_lemmatizer.lemmatize)

In [6]:
test_df['text'] = test_df['text'].apply(wordnet_lemmatizer.lemmatize)

In [67]:
tfidf = CountVectorizer()
tfidf.fit(train_df.text.append(test_df.text))
train_features = tfidf.transform(train_df.text)
le  = LabelEncoder()
le.fit(train_df.drug.append(test_df.drug))
train_m  = le.transform(train_df.drug)
train_m = train_m.reshape(5279,1)
newfeatures = hstack((train_features,train_m)).tocsr()

In [60]:
x,y = newfeatures,train_df.sentiment

In [68]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=.20, random_state=0)
for train_index, test_index in sss.split(x, y):
    x_train,x_test,y_train,y_test = x[train_index],x[test_index],y[train_index],y[test_index]

In [70]:
xgb.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [71]:
y_pred = xgb.predict(x_test)

In [72]:
f1_score(y_test,y_pred,average='macro')

0.30889260457433193